# Import

In [1]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [2]:
# comp_id = 380725    # 기업 고유 UID
p_num = 1      # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [3]:
Keyword = '삼성전자'

In [4]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
# print(search_req.text)

In [5]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text

In [6]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

[<a href="/Comp/CompSummary/380725">
                 삼성전자
             </a>]

In [7]:
comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [8]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode='
print(URL)

https://www.catch.co.kr/api/v1.0/comp/reviewInfo/380725/commentList?currentPage=1&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode=


# Json to DataFrame 

In [9]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
# print(raw_data)

In [10]:
data = raw_data['reviewList']
data[0]

{'idx': 209151,
 'CompID': '380725',
 'CompName': '삼성전자',
 'RegDate': '2023-08-10T08:41:40.253Z',
 'CI': '380725.jpg',
 'EmployText': '현직',
 'Gender2': 2,
 'RecomName': '추천',
 'EmployType': '정규직',
 'NewOld': '신입',
 'Answer': None,
 'Good': '어학  지원\r\n칼퇴  가능\r\n자울 출퇴근제\r\n식사 제공\r\n통근버스',
 'Bad': '수직적 조직문화\r\n선택적 복리후생\r\n불투명한 소통\r\n기타등등',
 'UsefulY': 0,
 'MyUsefulY': None,
 'MyOpinion': None,
 'CareerYear': 13,
 'CareerYearYN': 'Y',
 'Keyword1': None,
 'Keyword2': None,
 'Keyword3': None,
 'Keyword1YN': 'N',
 'Keyword2YN': 'N',
 'Keyword3YN': 'N',
 'JobName': '생산관리/공정관리/품질관리',
 'Area': '경북',
 'MyStarScore': 4}

In [11]:
df = pd.DataFrame(data)
df

,idx,CompID,CompName,RegDate,CI,EmployText,Gender2,RecomName,EmployType,NewOld,...,CareerYearYN,Keyword1,Keyword2,Keyword3,Keyword1YN,Keyword2YN,Keyword3YN,JobName,Area,MyStarScore
0,209151,380725,삼성전자,2023-08-10T08:41:40.253Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,생산관리/공정관리/품질관리,경북,4.0
1,209113,380725,삼성전자,2023-08-09T03:31:28.130Z,380725.jpg,전직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,생산/제조/설비/조립,경기,3.0
2,208104,380725,삼성전자,2023-06-26T05:25:22.170Z,380725.jpg,전직,2,추천,정규직,신입,...,N,None,None,None,N,N,N,웹개발,서울,4.6
3,208095,380725,삼성전자,2023-06-26T00:15:57.717Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,일반영업,서울,2.6
4,207978,380725,삼성전자,2023-06-21T05:23:01.440Z,380725.jpg,전직,2,비추,정규직,경력,...,Y,None,None,None,N,N,N,기획/전략/경영,경기,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,2020-09-26T11:23:26.790Z,380725.jpg,전직,2,추천,정규직,경력,...,N,None,None,None,N,N,N,판매/캐셔/매장관리,None,4.2
302,120816,380725,삼성전자,2020-09-24T05:02:07.343Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,기획/전략/경영,None,4.0
303,120504,380725,삼성전자,2020-09-12T07:34:44.560Z,380725.jpg,전직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,재무/세무/IR,None,4.0
304,120412,380725,삼성전자,2020-09-07T12:49:14.490Z,380725.jpg,전직,2,추천,정규직,경력,...,Y,None,None,None,N,N,N,네트워크/서버/보안,None,3.4


In [12]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'CI', 'EmployText', 'Gender2',
       'RecomName', 'EmployType', 'NewOld', 'Answer', 'Good', 'Bad', 'UsefulY',
       'MyUsefulY', 'MyOpinion', 'CareerYear', 'CareerYearYN', 'Keyword1',
       'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN',
       'JobName', 'Area', 'MyStarScore'],
      dtype='object')

In [13]:
df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',  'MyUsefulY', 
            'MyOpinion', 'CareerYear', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)

df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'RecomName',
       'Good', 'Bad', 'JobName', 'MyStarScore'],
      dtype='object')

In [14]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df

,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,2023-08-10T08:41:40.253Z,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,2023-08-09T03:31:28.130Z,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,2023-06-26T05:25:22.170Z,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,2023-06-26T00:15:57.717Z,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,2023-06-21T05:23:01.440Z,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,2020-09-26T11:23:26.790Z,전직,추천,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,2020-09-24T05:02:07.343Z,현직,추천,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,2020-09-12T07:34:44.560Z,전직,추천,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,2020-09-07T12:49:14.490Z,전직,추천,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


In [15]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력 (YYYYMMDD 형식의 날짜 표기)
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
df

,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,추천,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,추천,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,추천,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,추천,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [16]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=false&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',  'MyUsefulY', 
            'MyOpinion', 'CareerYear', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())


df

,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
199,148439,380725,삼성전자,20211020,전직,비추,돈을 많이 주고 이직에 최소한 경력으로 도움이 된다 복지는 좋다,돈을 많이 주는만큼 일을 많이 시키고 유연근무제도 안유연하고 야근이 많다,반도체/디스플레이,2.4
200,148244,380725,삼성전자,20211020,현직,추천,"자울출퇴근제이고, 합의된 비율로 재택근무가능하다. 예전에 비해 조직 분위기가 강압적...",제품이 너무 많고 일정이 타이트하여 개발자들과 관련부서(검증 등)에 로드가 심하다....,전기/전자/제어,4.0
201,147914,380725,삼성전자,20211020,현직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 연차를 쓸 수 있다.,가끔 업무강도가 너무 높고 야근이 갑자기 생겨서 불편하다.,사무/총무/법무,4.6
202,145509,380725,삼성전자,20210625,전직,추천,업무문화가 계속해서 변화를 시도해서 좋다. 뭔가 비젼을 제시하고 있어서 좋다,정년을 보장받기는 어렵다. 정리해고 할때는 가차없이 내보내고 노조가 없어서 아무소리...,전기/전자/제어,3.0


# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [17]:
# len(requests.get(url).json()['reviewList'])

In [18]:
# Catch 사이트에 검색을 하면, 최상위 검색결과의 comp_ui를 받아오기

comp_name = '삼성전자'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp_name)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [19]:
# 위에서 받아온 comp_id를 가지고 데이터 받아와서 데이터프레임으로 만들기

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))      # = 리스트 컴프리헨션
    df = pd.DataFrame(data)


    df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',  'MyUsefulY', 
                'MyOpinion', 'CareerYear', 'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
                axis=1, inplace=True)
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df


,idx,CompID,CompName,RegDate,EmployText,RecomName,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,추천,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,추천,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,추천,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,비추,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,추천,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,추천,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,추천,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,추천,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [20]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'RecomName',
       'Good', 'Bad', 'JobName', 'MyStarScore'],
      dtype='object')

In [21]:
df.rename(columns=
            {'idx':'review_uid', 
            'CompID':'comp_uid',
            'CompName':'comp_name', 
            'RegDate':'review_date', 
            'EmployText':'is_office', 
            'Good':'review_pos', 
            'Bad':'review_neg', 
            'MyStarScore':'review_rate', 
            'JobName':'position'}, 
            inplace=True)

df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,RecomName,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,현직,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0


### 전/현직 여부를 Boolean 값으로 변경

In [22]:
df[['is_office']]

,is_office
0,현직
1,전직
2,전직
3,현직
4,전직
...,...
301,전직
302,현직
303,전직
304,전직


In [23]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')
df[['is_office']]

,is_office
0,True
1,False
2,False
3,True
4,False
...,...
301,False
302,True
303,False
304,False


### 임의로 넣은 review_senti    <<- 모델 완성 후 교체 및 수정 요망 !!!

In [24]:
# 컬럼명 'RecomName'을 'review_senti'로 변경    <-- Catch에서 지원하는 추천/비추천 여부
df.rename(columns={'RecomName':'review_senti'}, inplace=True)
df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,True,추천,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0


In [25]:
# 추천/비추천 여부의 값을 1 | 0  으로 변환
df['review_senti'] = df['review_senti'].replace(['추천', '비추'], [1, 0])
df

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,True,1,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,False,1,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,False,1,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,True,1,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,False,0,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# 긍/부정 리뷰들을 한 컬럼에 합치기

In [43]:
a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

new_df = pd.concat([a, b])

new_df

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_cont,position,review_rate
0,209151,380725,삼성전자,20230810,True,1,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,False,1,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,False,1,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,True,1,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,False,0,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,네트워크/서버/보안,3.4


# DataFrame to CSV file

In [26]:
SAVE_PATH = r'./data/'

In [27]:
df['comp_name'][0]

'삼성전자'

In [28]:
comp = df['comp_name'][0]

In [29]:
# 디렉토리 생성 (이미 존재하면 생성하지 않음)

os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_catch.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

In [30]:
pd.read_csv(save_file_path)

,review_uid,comp_uid,comp_name,review_date,is_office,review_senti,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,20230810,True,1,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,False,1,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,False,1,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,True,1,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,False,0,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# 함수화

In [31]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

In [32]:
SAVE_PATH = r'./data/'
# SAVE_PATH = r'/app/data/reviews/'     <--- for Docker

In [33]:
re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))

'[<ahref="/Comp/CompSummary/380725">삼성전자</a>]'

In [34]:
re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

'삼성전자'

In [47]:
def RV_Catch(comp = str): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
    comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]




    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['idx', 'CompID', 'CompName', 'CI', 'Gender2', 'Answer', 'UsefulY', 'CareerYearYN',  'MyUsefulY', 'MyOpinion', 
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 
                'EmployType', 'NewOld', 'CareerYear', 'Area'], 
                axis=1, inplace=True)
        

        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')

        df['RecomName'] = df['RecomName'].replace(['추천', '비추'], [1, 0])


        df.rename(columns=
           {'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position',
           'RecomName':'review_senti'       # 임의로 쓰는 값 (나중에 모델 돌려서 나온 결과 값으로 대체 할 것.)
           }, inplace=True)



        # 긍/부정 리뷰들만 찢어서 합치기

        a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
        b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

        new_df = pd.concat([a, b])

        # return new_df




        # csv 파일로 저장.

        os.makedirs(SAVE_PATH, exist_ok=True)

        file_name = f"{comp_name}_catch.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)

        new_df.to_csv(save_file_path, index=False, encoding = "utf-8")
        
        return pd.read_csv(save_file_path)


        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [53]:
RV_Catch('SK하이')

,review_date,is_office,review_senti,review_cont,position,review_rate
0,20230722,False,1,업무량이 꽤 있는 편이라고 생각합니다. 워라밸 측면에서는 좀 아쉬움,기획/전략/경영,3.2
1,20230620,True,1,1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. ...,반도체/디스플레이,5.0
2,20230611,True,1,다만 제조기술이 아닌팀 그리고 일부 팀들은 야근이 매우 많은팀들이 있긴하다. 추가로...,생산/제조/설비/조립,3.8
3,20230609,True,1,보수적인 상사분들이 있다. (보통 나이가 많으신 분들 중에) 유연근무제이긴 하나 부...,생산/제조/설비/조립,4.2
4,20230608,True,1,서울 경기권에서 지역 마다 다르지만 출퇴근 시간이 오래걸린다. 조직 문화가 다소 경...,생산관리/공정관리/품질관리,4.4
...,...,...,...,...,...,...
167,20201118,True,1,"대기업인만큼 복지가 잘 되어있음, 하루 네끼 무료",반도체/디스플레이,4.4
168,20201118,True,1,반도체가 호황이라 돈을 많이 주지만 일하는 만큼 받는건지는 모르겠음,반도체/디스플레이,3.8
169,20201117,False,1,복지 좋음. 식사 시설 등 제공 좋습니다. 만족해요 다양한 선택권,반도체/디스플레이,4.0
170,20201010,True,1,복지혜택이 많다. 월급이 많다. 성과급이 많다,반도체/디스플레이,5.0


In [37]:
RV_Catch('화인석재')

리뷰가 존재하지 않습니다.


In [49]:
RV_Catch('삼성전자')

,review_date,is_office,review_senti,review_cont,position,review_rate
0,20230810,True,1,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,20230809,False,1,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,20230626,False,1,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,20230626,True,1,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,20230621,False,0,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...
607,20200926,False,1,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,판매/캐셔/매장관리,4.2
608,20200924,True,1,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,기획/전략/경영,4.0
609,20200912,False,1,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,재무/세무/IR,4.0
610,20200907,False,1,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,네트워크/서버/보안,3.4


In [56]:
RV_Catch('네이버')

,review_date,is_office,review_senti,review_cont,position,review_rate
0,20230623,False,1,다른 테크 회사들 대비해서는 조금 더 경직된 구조이긴 함. 아무래도 대기업이니,기획/전략/경영,3.6
1,20230606,True,1,"가끔 업무강도가 높음, 재택의 영향으로 직원들 얼굴 볼 기회가 적음, 성과 내기가 어려움",웹개발,5.0
2,20230527,False,1,업무량이 많아서 주말에 일을 할 경우가 있다 그런 거 말고는 딱히 없다,B2B영업/기술영업,4.2
3,20230518,False,0,"젊꼰이 많다 높은 업무강도 책임감등이 필요하다,,,,,",광고/시각디자인,3.4
4,20230517,True,1,업무강도가 높고 항상 일이 많다. 다들 열심히 해서 나도 잘해야할 것 같은 압박이 있다.,기획/전략/경영,4.0
...,...,...,...,...,...,...
81,20201128,True,1,"복지가 최고다. 회사 네임벨류 덕분에 스스로 자부심을 느낄 수 있다. 또한, 팀원들...",마케팅/PR/분석,3.8
82,20201127,False,1,업계 최고 기업 중 하나라서 이직 시에 유리하다.,콘텐츠/사이트운영,4.8
83,20201119,True,1,"네이버는 좋은 회사입니다, 회사 기업문화도 좋습니다",디자인기타,5.0
84,20201118,False,1,사내복지와 분위기 등 직원 복지부분은 준비가 매우 잘 되어있음,네트워크/서버/보안,4.4


In [55]:
RV_Catch('카카오')

,review_date,is_office,review_senti,review_cont,position,review_rate
0,20230728,True,1,경영진의 문제가 심각 일단 사이즈는 큰데 앞으로 뭐먹고..?,웹기획/PM/웹마케팅,3.8
1,20230609,True,1,이쪽 특이긴 하지만 연봉 인상률이 낮음 자유로운 분위기가 독이 되는게 본인이 자율적...,응용프로그램개발,3.6
2,20230602,True,1,진입장벽이 있고 업계 전반적으로 성장가능성이 높음. 연봉인상률 양호함. 부서에 따라...,빅데이터/AI,4.6
3,20230526,False,1,업무강도가 있는편이고 개인적인 업무 편향으로 먼가 혼자 일한다,마케팅/PR/분석,5.0
4,20230525,True,1,업무에 체계가 없어서 지시도 명확하지 않고..내가 무슨일을 하고 있는거지 하는 확신...,인사/노무/교육,3.6
...,...,...,...,...,...,...
83,20201118,False,1,자유로운 분위기입니다. 복장자유라서 편하게 다닐 수 있습니다,마케팅/PR/분석,3.8
84,20201118,True,1,"수평적이고 자기주도적인 업무 문화, 자유로운 분위기, 출퇴근 자유",사무/총무/법무,3.4
85,20201118,True,1,대기업이라 만족하고 좋아요 안정적이네요ㅎㅎ,통신기술/네트워크구축,4.0
86,20201117,True,1,복지시스템이 엄청 잘되있어서 자존감이 높아져요. 복장자유가 너무 좋아요,마케팅/PR/분석,5.0


In [57]:
from new_catch import RV_Catch

In [54]:
RV_Catch('NC소프트')

,review_date,is_office,review_senti,review_cont,position,review_rate
0,20230530,False,1,월급에 비해 120프로 일 시키는 곳.. 눈치 많이 봐야하고 정치질도 있지..,요리/영양/제과제빵/바리스타,3.4
1,20230528,True,0,"가족회사라 한명에 의해서 쉽게 의사결정바뀌고, 쓸데없는 임원이많다",일반영업,2.8
2,20230518,True,1,애들 때문에 일하면서 화가 나고 스트레스받아서 너무 지칩니다,AS/서비스/수리,4.6
3,20230517,False,0,서비스업에 근무하였는데 매출이 안나온다고 직원을 구해주지않아 출근하면 연장근무는 3...,외식/식음료,3.2
4,20230516,True,0,바쁜 점포 걸리면 워라벨 저리 가시오 이상한 팀장 만나면 손익 노이로제 걸림 구닥다...,고객관리/인바운드,3.0
5,20230417,True,0,성장하는 느낌이 전혀 들지 않음. 고일대로 고인 회사. 발전하려고 하지 않는다.,사무/총무/법무,1.2
6,20230329,False,0,대책없는 경영진 노답. 오너 리스크. 연봉이 너무 낮으며 인센티브가 너무 적은편으로...,인사/노무/교육,3.6
7,20230326,True,0,경영진이 도대체 뭘 원하는지 모르겠습니다. 점점 퇴보하고 있습니다,일반영업,3.2
8,20221229,False,0,주주 눈치보는 경영진. 주주가 최악임. 성과급은 없다고 보면됨.,재무/세무/IR,3.4
9,20221010,True,1,"경영진 능력부족, 일잘하는 사람만 계속시키는 불합리한 구조, 성과급은 개인의 능력이...",요리/영양/제과제빵/바리스타,3.2
